In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=713c997418f382e102f81fa605109666f4910afd0cfda028e5da47c9c4344825
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.32 ,random_state=0 , l1_ratio=0.9795, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_15_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10, 20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=323029, Sat Apr 18 19:34:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.873826641717305e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.714476420003242e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2
    3     57 1.685198051495844e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [21189.09426218] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [19018.47652115] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   38    722 4.725009805692787e+03 1.1e+00 1.86e+00  2e+00  2e+00 0:03.4
   64   1216 3.226355649957859e+03 1.1e+00 1.61e+00  2e+00  2e+00 0:07.4
  100   1900 -2.157679379316520e+03 1.2e+00 1.31e+00  1e+00  1e+00 0:11.4
  171   3249 -1.888785288535406e+04 1.2e+00 6.92e-01  7e-01  7e-01 0:17.4
  200   3800 -2.197266271093135e+04 1.2e+00 4.97e-01  5e-01  5e-01 0:19.9
  296   5624 -2.912560895363382e+04 1.2e+00 1.67e-01  2e-01  2e-01 0:27.9
  300   5700 -2.903817945892867e+04 1.2e+00 1.60e-01  2e-01  2e-01 0:28.2
  400   7600 -2.969070277746161e+04 1.2e+00 5.39e-02  5e-02  5e-02 0:36.6
  500   9500 -2.976618968081127e+04 1.2e+00 1.85e-02  2e-02  2e-02 0:45.0
  600  11400 -2.977502015234093e+04 1.2e+00 5.99e-03  6e-03  6e-03 0:53.3
  700  13300 -2.977592291741116e+04 1.2e+00 2.02e-03  2e-03  2e-03 1:01.7
  800  15200 -2.977604147751706e+04 1.2e+00 6.97e-04  7e-04  7e-04 1:10.1
  900  17100 -2.977605486860994e+04 1.3e+00 2.43e-04  2e-04  2e-04 1:18.1
 1000  19000 -2.977605628454046e+04 1.3e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [20084.57006765] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [18740.62801975] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.713878870601673e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.591365221139354e+04 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3


...

    3     57 1.537993356862315e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.2
   40    760 5.387286070715410e+03 1.1e+00 1.92e+00  2e+00  2e+00 0:03.3
   89   1691 3.425110849794004e+03 1.2e+00 1.59e+00  2e+00  2e+00 0:07.3
  100   1900 -5.719692446930021e+02 1.2e+00 1.46e+00  1e+00  1e+00 0:08.2
  174   3306 -1.651728305248367e+04 1.2e+00 7.75e-01  8e-01  8e-01 0:14.2
  200   3800 -2.129886237092139e+04 1.2e+00 5.69e-01  6e-01  6e-01 0:16.4
  299   5681 -2.872225616769350e+04 1.2e+00 1.96e-01  2e-01  2e-01 0:24.4
  300   5700 -2.875734116066295e+04 1.2e+00 1.94e-01  2e-01  2e-01 0:24.5
  400   7600 -2.964521186521753e+04 1.2e+00 6.54e-02  6e-02  7e-02 0:32.5
  500   9500 -2.976569567604636e+04 1.2e+00 2.21e-02  2e-02  2e-02 0:40.6
  600  11400 -2.977469519969208e+04 1.2e+00 6.82e-03  7e-03  7e-03 0:48.7
  700  13300 -2.977591043585251e+04 1.2e+00 2.31e-03  2e-03  2e-03 0:56.8
  800  15200 -2.977603745615118e+04 1.2e+00 8.28e-04  8e-04  8e-04 1:04.8
  900  17100 -2.977605472189179e+04 1.2e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [20084.57006765] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [21460.17007337] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.749689242563523e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2
    3     57 1.581002793811422e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [22665.19808529] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   35    665 2.949880516662050e+03 1.1e+00 1.85e+00  2e+00  2e+00 0:03.4
   77   1463 -3.878438606555837e+03 1.1e+00 1.45e+00  1e+00  1e+00 0:07.4
  100   1900 -5.673906309039472e+03 1.2e+00 1.26e+00  1e+00  1e+00 0:09.6
  163   3097 -1.231838906528596e+04 1.2e+00 8.45e-01  8e-01  8e-01 0:15.7
  200   3800 -1.542918860299092e+04 1.2e+00 6.47e-01  6e-01  7e-01 0:19.2
  283   5377 -1.995096762253824e+04 1.2e+00 3.79e-01  4e-01  4e-01 0:27.2
  300   5700 -2.042450871182740e+04 1.2e+00 3.36e-01  3e-01  3e-01 0:28.8
  400   7600 -2.252418156577439e+04 1.2e+00 1.78e-01  2e-01  2e-01 0:38.4
  500   9500 -2.354736483461414e+04 1.3e+00 1.24e-01  1e-01  1e-01 0:48.0
  600  11400 -2.406370328094518e+04 1.5e+00 9.76e-02  9e-02  1e-01 0:57.7
  700  13300 -2.448955941553721e+04 1.7e+00 7.32e-02  7e-02  8e-02 1:07.3
  800  15200 -2.472796848142910e+04 1.9e+00 5.67e-02  5e-02  7e-02 1:16.9
  900  17100 -2.482341464339101e+04 2.0e+00 3.86e-02  4e-02  5e-02 1:26.5
 1000  19000 -2.491865111612391e+04 2.1

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F15_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:200])
    SVM_Fun [i] = F15(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F15(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([3719.67617267]), count=array([1])) 7.9384249543084e-05
SVM
ModeResult(mode=array([18845.40764033]), count=array([1])) 3298.1261124998373
ELN
ModeResult(mode=array([25066.96432312]), count=array([1])) 5693.052773970632
